In [52]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from pathlib import Path
import regex as re
import nltk
from datetime import date
import twint
import nest_asyncio
import tensorflow as tf
nest_asyncio.apply()
nltk.download('stopwords')
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
# Set the random seed for reproducibility
from numpy.random import seed
seed(1)

from tensorflow import random
random.set_seed(2)

In [54]:
# Data crawling using Twint
import twint
import nest_asyncio
nest_asyncio.apply()

import datetime

# Fetch data in a example data frame: 2020-04-14 00:00:00 to 2020-04-14 00:30:00

start_time='2020-01-01 00:00:00'

c = twint.Config()
c.Search = "microsoft"
c.Lang="en"
c.Since= start_time
c.Limit = 5000
c.Pandas = True
c.Store_csv = True
c.Retries_count = 20
c.Hide_output = True
c.Custom["tweet"] = ["id", "date", "time", "tweet", "retweets_count", "likes_count"]
c.Resume = "./test.session"
c.Output = "./test.csv"
twint.run.Search(c)

In [55]:
file_path = 'test.csv'
df = pd.read_csv(file_path, parse_dates=[['date','time']], infer_datetime_format=True)
df.head()

,date_time,id,tweet,retweets_count,likes_count
0,2021-01-19 20:55:16,1351709850380587008,Microsoft partners with GM subsidiary Cruise o...,0,0
1,2021-01-19 20:55:03,1351709796534255617,Xbox support is so fucking useless thanks for ...,0,0
2,2021-01-19 20:54:55,1351709762405216258,Thank God I remembered Xbox has Microsoft Edge...,0,1
3,2021-01-19 20:54:55,1351709760047878144,Mana nak dapat Microsoft Access woii😫😫,0,0
4,2021-01-19 20:54:52,1351709749641965571,Anybody else wonder why noone has started a ma...,0,0


In [56]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 

analyzer = SentimentIntensityAnalyzer()
text_sent = {'Compound':[],
                 'Negative':[],
                 'Neutral':[],
                 'Positive':[]}
# Get sentiment from articles
for index, row in df.iterrows():
    try:
        text_sentiment = analyzer.polarity_scores(row["tweet"])
        text_sent["Compound"].append(text_sentiment["compound"])
        text_sent["Negative"].append(text_sentiment["neg"])
        text_sent["Neutral"].append(text_sentiment["neu"])
        text_sent["Positive"].append(text_sentiment["pos"])
        #text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass
# Add the newly created scores to the previously created bitcoin dataframe
text_sent_df = pd.DataFrame(text_sent)
test_sentiment = text_sent_df.join(df)
test_sentiment.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Compound,Negative,Neutral,Positive,date_time,id,tweet,retweets_count,likes_count
0,0.0000,0.000,1.000,0.000,2021-01-19 20:55:16,1351709850380587008,Microsoft partners with GM subsidiary Cruise o...,0,0
1,0.8612,0.108,0.574,0.319,2021-01-19 20:55:03,1351709796534255617,Xbox support is so fucking useless thanks for ...,0,0
2,0.7269,0.000,0.738,0.262,2021-01-19 20:54:55,1351709762405216258,Thank God I remembered Xbox has Microsoft Edge...,0,1
3,0.0000,0.000,1.000,0.000,2021-01-19 20:54:55,1351709760047878144,Mana nak dapat Microsoft Access woii😫😫,0,0
4,-0.4753,0.072,0.928,0.000,2021-01-19 20:54:52,1351709749641965571,Anybody else wonder why noone has started a ma...,0,0


In [57]:
test_sentiment['Sentiment_Score'] = np.nan
test_sentiment.loc[test_sentiment['Compound'] >= 0.05, 'Sentiment_Score'] = 1
test_sentiment.loc[test_sentiment['Compound'] <= - 0.05, 'Sentiment_Score'] = -1
test_sentiment.loc[(test_sentiment['Compound'] < 0.05) & (test_sentiment['Compound'] > -0.05), 'Sentiment_Score']= 0
test_sentiment.head()

,Compound,Negative,Neutral,Positive,date_time,id,tweet,retweets_count,likes_count,Sentiment_Score
0,0.0000,0.000,1.000,0.000,2021-01-19 20:55:16,1351709850380587008,Microsoft partners with GM subsidiary Cruise o...,0,0,0.0
1,0.8612,0.108,0.574,0.319,2021-01-19 20:55:03,1351709796534255617,Xbox support is so fucking useless thanks for ...,0,0,1.0
2,0.7269,0.000,0.738,0.262,2021-01-19 20:54:55,1351709762405216258,Thank God I remembered Xbox has Microsoft Edge...,0,1,1.0
3,0.0000,0.000,1.000,0.000,2021-01-19 20:54:55,1351709760047878144,Mana nak dapat Microsoft Access woii😫😫,0,0,0.0
4,-0.4753,0.072,0.928,0.000,2021-01-19 20:54:52,1351709749641965571,Anybody else wonder why noone has started a ma...,0,0,-1.0


In [58]:
# Convert [-1, 0, 1] to [negative, neutral, and positive]

# Negative
test_sentiment.loc[test_sentiment['Sentiment_Score'] == -1, 'Sentiment_Score'] = 'Negative'

# Neutral

test_sentiment.loc[test_sentiment['Sentiment_Score'] == 0, 'Sentiment_Score'] = 'Neutral'

# Postive
test_sentiment.loc[test_sentiment['Sentiment_Score'] == 1, 'Sentiment_Score'] = 'Positive'

# View Into DataFrame
test_sentiment.tail()

,Compound,Negative,Neutral,Positive,date_time,id,tweet,retweets_count,likes_count,Sentiment_Score
55650,0.0000,0.0,1.000,0.000,2021-01-15 13:48:00,1350152772461551616,The Store in @Microsoft #Windows needs work. T...,0,2,Neutral
55651,0.4767,0.0,0.829,0.171,2021-01-15 13:47:57,1350152760067567616,@Dr_Paul_Penn Imagine the money Microsoft coul...,0,0,Positive
55652,0.0000,0.0,1.000,0.000,2021-01-15 13:47:50,1350152730388512768,@Microsoft HELLO @Microsoft How are you !!,0,0,Neutral
55653,0.0000,0.0,1.000,0.000,2021-01-15 13:47:49,1350152726848663553,Microsoft schließt 83 Sicherheitslücken zum er...,0,0,Neutral
55654,0.0000,0.0,1.000,0.000,2021-01-15 13:47:48,1350152722453032967,@linoano Essa lista vai deixar bastante gente ...,0,1,Neutral


In [59]:
# Clean the Text Function
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')


def clean_text(text):
  text = text.lower() # lowercase text
  text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
  text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
  text = text.replace('x', '')
 # text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
  return text

In [60]:
# Apply the clean text function above
test_sentiment['tweet'] = test_sentiment['tweet'].astype(str)
test_sentiment['tweet'] = test_sentiment['tweet'].apply(clean_text)
test_sentiment['tweet'] = test_sentiment['tweet'].str.replace('\d+', '')
test_sentiment.head()

,Compound,Negative,Neutral,Positive,date_time,id,tweet,retweets_count,likes_count,Sentiment_Score
0,0.0000,0.000,1.000,0.000,2021-01-19 20:55:16,1351709850380587008,microsoft partners with gm subsidiary cruise o...,0,0,Neutral
1,0.8612,0.108,0.574,0.319,2021-01-19 20:55:03,1351709796534255617,bo support is so fucking useless thanks for th...,0,0,Positive
2,0.7269,0.000,0.738,0.262,2021-01-19 20:54:55,1351709762405216258,thank god i remembered bo has microsoft edge a...,0,1,Positive
3,0.0000,0.000,1.000,0.000,2021-01-19 20:54:55,1351709760047878144,mana nak dapat microsoft access woii,0,0,Neutral
4,-0.4753,0.072,0.928,0.000,2021-01-19 20:54:52,1351709749641965571,anybody else wonder why noone has started a ma...,0,0,Negative


In [61]:
test_sentiment.describe()

,Compound,Negative,Neutral,Positive,id,retweets_count,likes_count
count,55655.000000,55655.000000,55655.000000,55655.000000,5.565500e+04,55655.000000,55655.000000
mean,0.113517,0.033997,0.890691,0.075310,1.350973e+18,1.025856,4.533285
std,0.372372,0.077782,0.136093,0.114958,4.972912e+14,18.451687,74.997074
min,-0.978100,0.000000,0.087000,0.000000,1.350153e+18,0.000000,0.000000
25%,0.000000,0.000000,0.811000,0.000000,1.350491e+18,0.000000,0.000000
50%,0.000000,0.000000,0.937000,0.000000,1.351062e+18,0.000000,0.000000
75%,0.380200,0.000000,1.000000,0.130000,1.351451e+18,0.000000,1.000000
max,0.990100,0.813000,1.000000,0.913000,1.351710e+18,3298.000000,6896.000000


In [62]:
# Convert [-1, 0, 1] to [negative, neutral, and positive]

# Negative
test_sentiment.loc[test_sentiment['Sentiment_Score'] == -1, 'Sentiment_Score'] = 'Negative'

# Neutral

test_sentiment.loc[test_sentiment['Sentiment_Score'] == 0, 'Sentiment_Score'] = 'Neutral'

# Postive
test_sentiment.loc[test_sentiment['Sentiment_Score'] == 1, 'Sentiment_Score'] = 'Positive'

# View Into DataFrame
test_sentiment.tail()

,Compound,Negative,Neutral,Positive,date_time,id,tweet,retweets_count,likes_count,Sentiment_Score
55650,0.0000,0.0,1.000,0.000,2021-01-15 13:48:00,1350152772461551616,the store in microsoft #windows needs work th...,0,2,Neutral
55651,0.4767,0.0,0.829,0.171,2021-01-15 13:47:57,1350152760067567616,dr_paul_penn imagine the money microsoft coul...,0,0,Positive
55652,0.0000,0.0,1.000,0.000,2021-01-15 13:47:50,1350152730388512768,microsoft hello microsoft how are you,0,0,Neutral
55653,0.0000,0.0,1.000,0.000,2021-01-15 13:47:49,1350152726848663553,microsoft schliet sicherheitslcken zum ersten...,0,0,Neutral
55654,0.0000,0.0,1.000,0.000,2021-01-15 13:47:48,1350152722453032967,linoano essa lista vai deiar bastante gente d...,0,1,Neutral


In [63]:
# Clean the Text Function
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')


def clean_text(text):
  text = text.lower() # lowercase text
  text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
  text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
  text = text.replace('x', '')
 # text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
  return text

In [64]:
# Apply the clean text function above
test_sentiment['tweet'] = test_sentiment['tweet'].astype(str)
test_sentiment['tweet'] = test_sentiment['tweet'].apply(clean_text)
test_sentiment['tweet'] = test_sentiment['tweet'].str.replace('\d+', '')
test_sentiment["class"] = test_sentiment["Compound"].apply(lambda x: 1 if x >= 0 else 0)
test_sentiment.head()

,Compound,Negative,Neutral,Positive,date_time,id,tweet,retweets_count,likes_count,Sentiment_Score,class
0,0.0000,0.000,1.000,0.000,2021-01-19 20:55:16,1351709850380587008,microsoft partners with gm subsidiary cruise o...,0,0,Neutral,1
1,0.8612,0.108,0.574,0.319,2021-01-19 20:55:03,1351709796534255617,bo support is so fucking useless thanks for th...,0,0,Positive,1
2,0.7269,0.000,0.738,0.262,2021-01-19 20:54:55,1351709762405216258,thank god i remembered bo has microsoft edge a...,0,1,Positive,1
3,0.0000,0.000,1.000,0.000,2021-01-19 20:54:55,1351709760047878144,mana nak dapat microsoft access woii,0,0,Neutral,1
4,-0.4753,0.072,0.928,0.000,2021-01-19 20:54:52,1351709749641965571,anybody else wonder why noone has started a ma...,0,0,Negative,0


In [86]:
# Define Variables
from sklearn.metrics import accuracy_score
X = test_sentiment[['Compound','Positive','retweets_count','likes_count','Neutral']]
X = X.reshape(X.shape[1:])
y = test_sentiment["class"]

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [83]:
X.shape

(5, 55655)

In [84]:
y.shape

(55655,)

In [85]:
# Splitting 
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.33, random_state = 7)

ValueError: Found input variables with inconsistent numbers of samples: [5, 55655]

In [71]:
# Instantiation 
xgb_r = xg.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 10, seed = 123)

In [72]:
# Fitting the model 
xgb_r.fit(train_X, train_y)

[22:28:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=8, num_parallel_tree=1,
             objective='reg:linear', random_state=123, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=123, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [73]:
# Predict the model 
predictions = xgb_r.predict(test_X)

In [74]:
# RMSE Computation 
rmse = np.sqrt(MSE(test_y, pred)) 
print("RMSE : % f" %(rmse))

ValueError: Found input variables with inconsistent numbers of samples: [18367, 16717]

In [75]:
accuracy = accuracy_score(test_y, predictions)
print(accuracy)

ValueError: Classification metrics can't handle a mix of binary and continuous targets